In [4]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests




In [6]:
# perform Api calls 
api_key = "8469ce8835d24f5f832bfd64ce9d0e2e"
url="http://geoservices.tamu.edu/Services/Geocode/WebService/GeocoderWebServiceHttpNonParsedDetailed_V04_01.aspx?"
zip_code = 73301
city = "austin"
query_url = f"{url}appid={api_key}&city={city}"
response = requests.get(query_url).json
# payloads={streetAddress=9355%20Burton%20Way&
# city=Beverly%20Hills
# &state=ca&
# zip=90210&
#apikey=8469ce8835d24f5f832bfd64ce9d0e2e&format=json&census=true&censusYear=2000|2010&notStore=false&version=4.01"}
